In [9]:
import pandas as pd
import numpy as np
import random as random
random.seed(3)

In [10]:
random.random()

0.23796462709189137

## Plant Location  

#### Manufacturing variable costs

In [11]:
# Import Costs
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col = 0)
manvar_costs

USA  Germany  Japan  Brazil  India
Variable Costs ($/Unit)                                    
USA                       12       12     12      12     12
Germany                   13       13     13      13     13
Japan                     10       10     10      10     10
Brazil                     8        8      8       8      8
India                      5        5      5       5      5

In [12]:
time_cost = pd.read_excel('time_cost.xlsx', index_col = 0)

In [13]:
time_cost

USA  Germany  Japan  Brazil  India
Variable time (hours)                                    
USA                      0       10      8       2     32
Germany                 10        0     28       8     12
Japan                    8       28      0      10     14
Brazil                   2        8     10       0     18
India                   32       12     14      18      0

In [14]:
fresh = 30
fresh_time = time_cost.copy()
for i in fresh_time:
    for j in fresh_time:
        if i!=j:
            fresh_time[i][j] = fresh

In [15]:
fresh_time

USA  Germany  Japan  Brazil  India
Variable time (hours)                                    
USA                      0       30     30      30     30
Germany                 30        0     30      30     30
Japan                   30       30      0      30     30
Brazil                  30       30     30       0     30
India                   30       30     30      30      0

In [16]:
expire_time = time_cost - fresh_time
expire_time

USA  Germany  Japan  Brazil  India
Variable time (hours)                                    
USA                      0      -20    -22     -28      2
Germany                -20        0     -2     -22    -18
Japan                  -22       -2      0     -20    -16
Brazil                 -28      -22    -20       0    -12
India                    2      -18    -16     -12      0

#### Freight costs

In [17]:
# Import Costs
freight_costs = pd.read_excel('freight_costs.xlsx', index_col = 0)
freight_costs

USA  Germany  Japan  Brazil  India
Freight Costs ($/Container)                                      
USA                              0    12250   1100   16100   8778
Germany                      13335        0   8617   20244  10073
Japan                        15400    22750      0   43610  14350
Brazil                       16450    22050  28000       0  29750
India                        13650    15400  24500   29400      0

#### Variable Costs

In [18]:
# Variable Costs
var_cost = freight_costs/1000 + manvar_costs 

var_cost

USA  Germany   Japan  Brazil   India
Freight Costs ($/Container)                                         
USA                          12.000    24.25  13.100  28.100  20.778
Germany                      26.335    13.00  21.617  33.244  23.073
Japan                        25.400    32.75  10.000  53.610  24.350
Brazil                       24.450    30.05  36.000   8.000  37.750
India                        18.650    20.40  29.500  34.400   5.000

#### Fixed Costs

In [19]:
# Import Costs
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
fixed_costs

Low  High
USA      6500  9500
Germany  4980  7270
Japan    6230  9100
Brazil   3230  4730
India    2110  6160

#### Plants Capacity

In [20]:
# Two types of plants: Low Capacity and High Capacity Plant
cap = pd.read_excel('capacity.xlsx', index_col = 0)
cap

Low  High
Capacity (kUnits/month)           
USA                      500  1500
Germany                  500  1500
Japan                    500  1500
Brazil                   500  1500
India                    500  3000

In [21]:
cap['Low']['USA']

500

#### Demand 

In [22]:
# -- Demand
demand = pd.read_excel('demand.xlsx')

In [23]:
demand

(Units/month)   Demand
0           USA  2800000
1       Germany    90000
2         Japan  1700000
3        Brazil   145000
4         India   160000

In [24]:
tot_demand = 0
for i in demand['Demand']:
    tot_demand += i
tot_demand

4895000

In [25]:
demand_center = []
demand_qnty = []
for i, j in zip(demand['(Units/month)'], demand['Demand']):
    print(i)
    demand_center.append(i)
    demand_qnty.append(j)

USA
Germany
Japan
Brazil
India


In [26]:
prdn_center = []
prdn_qnty = []
for i in demand_center:
    for j in ['Low', 'High']:
        prdn_center.append(j+' '+i)
        prdn_qnty.append(cap[j][i])

In [27]:
prdn_center, prdn_qnty

(['Low USA',
  'High USA',
  'Low Germany',
  'High Germany',
  'Low Japan',
  'High Japan',
  'Low Brazil',
  'High Brazil',
  'Low India',
  'High India'],
 [500, 1500, 500, 1500, 500, 1500, 500, 1500, 500, 3000])

In [28]:
loc_seq = random.sample([i for i in range(len(demand_center))], k=len(demand_center))

In [29]:
loc_seq

[4, 1, 3, 2, 0]

In [105]:
out = get_palnts_prod()
print(out)
np.sum(out[1])

(['Low India', 'High Japan', 'Low USA', 'Low Brazil', 'High Germany', 'Low Japan'], [500000, 1500000, 500000, 500000, 1500000, 395000])


4895000

In [106]:
dmnd_cntr = []
dmnd_cap = []
for i in out[0]:
    dmnd_cntr.append(i)
    dmnd_cap.append(1000*prdn_qnty[prdn_center.index(i)])

In [107]:
dmnd_cntr, dmnd_cap

(['Low India',
  'High Japan',
  'Low USA',
  'Low Brazil',
  'High Germany',
  'Low Japan'],
 [500000, 1500000, 500000, 500000, 1500000, 500000])

In [259]:
# function to select the initial plants and their production
def get_palnts_prod():
    global tot_demand, prdn_center, prdn_qnty
    net_cost = tot_demand
    temp = 0
    t = random.randint(0, 5)
    while(temp - net_cost<=0):
        temp = 0
        prod_site = []
        prod_qunatity = []
        prod_seq = random.sample([i for i in range(len(prdn_center))], k=len(prdn_center))
        for i in prod_seq:
            if (temp-net_cost<=0):
                prod_site.append(prdn_center[i])
                #temp_prod = random.randint(1, 100)*prdn_qnty[i]*10
                temp_prod = prdn_qnty[i]*1000
                prod_qunatity.append(temp_prod)
                temp += temp_prod
            else:
                prod_qunatity[-1] =  net_cost - (np.sum(prod_qunatity) - prod_qunatity[-1])
                break
    return prod_site, prod_qunatity

def cfactorial(x):
    if x==1:
        return 1
    else:
        return x*cfactorial(x-1)

def get_prod_status(demand, prod_list, prod_quantity):
    # please remove the site with zero balance first
    for i in range(len(prod_list)):
        if prod_quantity[i]<=0:
            prod_list.pop(i)
            prod_quantity.pop(i)
    l_c = []
    l_nc = []
    for i in range(len(prod_list)):
        if prod_quantity[i]>=demand:
            l_c.append(prod_list[i])
        else:
            l_nc.append(prod_list[i])
    return l_c, l_nc

def get_prod_site(demand, prod_list, prod_quantity):
    f_l_c, f_l_nc = get_prod_status(demand, prod_list, prod_quantity)
    l_c = []
    l_nc = []
    for i in f_l_c:
        l_c.append([i])
    for i in f_l_nc:
        l_nc.append([i])
    t = 0
    while len(l_nc)!=0:
        for i in l_nc:
            t+=1
#             print(t)
            temp_prod_list = prod_list.copy()
            temp_prod_quantity = prod_quantity.copy()
            temp_demand = demand
            for j in i:
                loc = temp_prod_list.index(j)
                temp_prod_list.pop(loc)
                temp_cost = temp_prod_quantity.pop(loc)
                temp_demand = temp_demand - temp_cost
            temp_l_c, temp_l_nc = get_prod_status(temp_demand, temp_prod_list.copy(), temp_prod_quantity.copy())
#             print(l_c, l_nc, 'sd\n\n')
#             print(i, 'i')
#             print(temp_l_c, temp_l_nc, 'cdcsd\n\n')
            for k in temp_l_c:
#                 print(i)
#                 print(i, 'to')
#                 print('temp1\n\n')
                t_0_2 = i.copy()
                t_0_2.append(k)
#                 print(t_0_2, 'final\n')
                l_c.append(t_0_2)
            for l in temp_l_nc:
                t_0_3 = i.copy()
                t_0_3.append(l)
                l_nc.append(t_0_3)
            l_nc.remove(i)
#             print(l_c, l_nc, 'ssadawdd\n\n')
        if t>=10000:
            print('Caution code not completed')
            break
    return l_c

In [271]:
out = get_prod_site(4895000, dmnd_cntr.copy(), dmnd_cap.copy())

In [272]:
len(out)

720

In [277]:
out

[['Low India',
  'High Japan',
  'Low Brazil',
  'High Germany',
  'Low Japan',
  'Low USA'],
 ['Low India',
  'High Japan',
  'Low Japan',
  'Low USA',
  'High Germany',
  'Low Brazil'],
 ['Low India',
  'High Japan',
  'Low Japan',
  'High Germany',
  'Low Brazil',
  'Low USA'],
 ['Low India',
  'Low Brazil',
  'Low USA',
  'High Germany',
  'Low Japan',
  'High Japan'],
 ['Low India',
  'Low Brazil',
  'Low Japan',
  'High Japan',
  'High Germany',
  'Low USA'],
 ['Low India',
  'Low Brazil',
  'Low Japan',
  'High Germany',
  'Low USA',
  'High Japan'],
 ['Low India',
  'Low Japan',
  'Low USA',
  'Low Brazil',
  'High Germany',
  'High Japan'],
 ['Low India',
  'Low Japan',
  'High Germany',
  'High Japan',
  'Low Brazil',
  'Low USA'],
 ['Low India',
  'Low Japan',
  'High Germany',
  'Low Brazil',
  'Low USA',
  'High Japan'],
 ['Low USA',
  'High Japan',
  'Low Brazil',
  'High Germany',
  'Low Japan',
  'Low India'],
 ['Low USA',
  'High Japan',
  'Low Japan',
  'Low India',
 

In [279]:
for i in out:
    itr = 0
    for j in i:
        print(j)
        itr+=dmnd_cap[dmnd_cntr.index(j)]
    print(itr-4895000, i)

Low India
High Japan
Low Brazil
High Germany
Low Japan
Low USA
105000 ['Low India', 'High Japan', 'Low Brazil', 'High Germany', 'Low Japan', 'Low USA']
Low India
High Japan
Low Japan
Low USA
High Germany
Low Brazil
105000 ['Low India', 'High Japan', 'Low Japan', 'Low USA', 'High Germany', 'Low Brazil']
Low India
High Japan
Low Japan
High Germany
Low Brazil
Low USA
105000 ['Low India', 'High Japan', 'Low Japan', 'High Germany', 'Low Brazil', 'Low USA']
Low India
Low Brazil
Low USA
High Germany
Low Japan
High Japan
105000 ['Low India', 'Low Brazil', 'Low USA', 'High Germany', 'Low Japan', 'High Japan']
Low India
Low Brazil
Low Japan
High Japan
High Germany
Low USA
105000 ['Low India', 'Low Brazil', 'Low Japan', 'High Japan', 'High Germany', 'Low USA']
Low India
Low Brazil
Low Japan
High Germany
Low USA
High Japan
105000 ['Low India', 'Low Brazil', 'Low Japan', 'High Germany', 'Low USA', 'High Japan']
Low India
Low Japan
Low USA
Low Brazil
High Germany
High Japan
105000 ['Low India', 'Low

In [118]:
a = ['ddc', 'dfdc', 'vex  rw']

In [119]:
a

['ddc', 'dfdc', 'vex  rw']

In [120]:
a.index('vex  rw')

2

In [52]:
a

['ddc', 'verw']

In [281]:
cfactorial(6)

720

(['High India', 'Low USA', 'High Brazil'], [3000000, 500000, 1395000])


4895000

In [39]:
random.sample([i for i in range(len(prdn_center))], k=len(prdn_center))

[6, 2, 3, 5, 1, 4, 7, 8, 0, 9]

In [40]:
len(prdn_center)

10

In [41]:
random.getrandbits(78)

289388061163530331516708

In [42]:
for i in range(10):
    print(random.randint(1, 100))

76
6
39
100
4
35
61
77
93
50


In [283]:
random.randint(10, 14)

12